# SwinT FineTuning with CIFAR-10

## Import Library

In [23]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from datasets import load_dataset,load_metric

from transformers import AutoImageProcessor, SwinForImageClassification, TrainingArguments, Trainer

## Load Dataset (CIFAR10)

In [24]:

dataset = load_dataset('cifar10',split='train')
dataset

Found cached dataset cifar10 (C:/Users/USER/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)


Dataset({
    features: ['img', 'label'],
    num_rows: 50000
})

In [25]:
label2id = {"plane":0, "car":1, "bird":2, "cat":3, "deer":4, "dog":5, "frog":6, "horse":7, "ship":8, "truck":9}
id2label = {0:"plane", 1:"car", 2:"bird", 3:"cat", 4:"deer", 5:"dog", 6:"frog", 7:"horse", 8:"ship", 9:"truck"}

In [26]:
model_name_or_path = 'microsoft/swin-tiny-patch4-window7-224'
feature_extractor = AutoImageProcessor.from_pretrained(model_name_or_path)

loading configuration file preprocessor_config.json from cache at C:\Users\USER/.cache\huggingface\hub\models--microsoft--swin-tiny-patch4-window7-224\snapshots\83d40fb5b9320b349382208d9e7fe998484e99df\preprocessor_config.json
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
size should be a dictionary on of the following set of keys: ({'height', 'width'}, {'shortest_edge'}, {'shortest_edge', 'longest_edge'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



In [7]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['img']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['label'] = example_batch['label']
    return inputs

In [8]:
prepared_ds = dataset.with_transform(transform)
prepared_ds = prepared_ds.train_test_split(test_size=0.1)
prepared_ds

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 5000
    })
})

## Training

In [9]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [10]:
metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

C:\Users\USER\AppData\Local\Temp/ipykernel_4328/2425197347.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [12]:

model = SwinForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=10,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes = True)

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = TrainingArguments(
  output_dir="./",
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=2,
  fp16=True,
  save_steps=10000,
  eval_steps=10000,
  logging_steps=1000,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  
)

PyTorch: setting up devices


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

Using cuda_amp half precision backend


In [17]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 45000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 11250
  Number of trainable parameters = 27527044
  1%|          | 100/11250 [01:12<1:56:14,  1.60it/s]

{'loss': 2.0563, 'learning_rate': 0.00019872000000000002, 'epoch': 0.02}


  2%|▏         | 200/11250 [02:14<1:55:45,  1.59it/s]

{'loss': 1.3852, 'learning_rate': 0.00019694222222222223, 'epoch': 0.04}


  3%|▎         | 300/11250 [03:17<1:54:51,  1.59it/s]

...

{'loss': 1.23, 'learning_rate': 3.2533333333333332e-06, 'epoch': 1.97}


100%|█████████▉| 11200/11250 [2:00:41<00:30,  1.61it/s]

{'loss': 1.2594, 'learning_rate': 1.4755555555555555e-06, 'epoch': 1.99}


100%|██████████| 11250/11250 [2:01:12<00:00,  1.61it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./checkpoint-10000 (score: 1.1885565519332886).
100%|██████████| 11250/11250 [2:01:12<00:00,  1.55it/s]
Saving model checkpoint to ./
Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin
Image processor saved in ./preprocessor_config.json


{'train_runtime': 7272.7674, 'train_samples_per_second': 12.375, 'train_steps_per_second': 1.547, 'train_loss': 1.129531569756402, 'epoch': 2.0}
***** train metrics *****
  epoch                    =        2.0
  train_loss               =     1.1295
  train_runtime            = 2:01:12.76
  train_samples_per_second =     12.375
  train_steps_per_second   =      1.547


In [20]:
dataset_test = load_dataset('cifar10',split='test')
dataset_test

Found cached dataset cifar10 (C:/Users/USER/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)


Dataset({
    features: ['img', 'label'],
    num_rows: 10000
})

## Testing

In [18]:
model = SwinForImageClassification.from_pretrained(
    './checkpoint-10000/',
    num_labels=10,
    id2label=id2label,
    label2id=label2id)

loading configuration file ./checkpoint-10000/config.json
Model config SwinConfig {
  "_name_or_path": "microsoft/swin-tiny-patch4-window7-224",
  "architectures": [
    "SwinForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    6,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 96,
  "encoder_stride": 32,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "plane",
    "1": "car",
    "2": "bird",
    "3": "cat",
    "4": "deer",
    "5": "dog",
    "6": "frog",
    "7": "horse",
    "8": "ship",
    "9": "truck"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "bird": 2,
    "car": 1,
    "cat": 3,
    "deer": 4,
    "dog": 5,
    "frog": 6,
    "horse": 7,
    "plane": 0,
    "ship": 8,
    "truck": 9
  },
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "swin",
  "num_channels": 3,
  "num_heads": [
    3,
    6,
    12,
    24
  ],
  "num_layers"

In [21]:
prepared_ds_test = dataset_test.with_transform(transform)

In [22]:
# Predictions
y_test_predict = trainer.predict(prepared_ds_test)

# Take a look at the predictions
y_test_predict

***** Running Prediction *****
  Num examples = 10000
  Batch size = 8
100%|██████████| 1250/1250 [05:01<00:00,  4.14it/s]


PredictionOutput(predictions=array([[-1.347e+00, -1.432e+00, -1.154e+00, ..., -1.906e+00, -1.865e+00,
        -2.371e+00],
       [ 1.724e-03,  1.224e-01, -1.603e+00, ..., -2.043e+00,  8.379e-01,
        -6.997e-01],
       [-6.089e-01,  8.643e-01, -1.550e+00, ..., -1.796e+00,  6.260e-01,
         1.650e-01],
       ...,
       [-1.830e+00, -1.491e+00, -5.156e-01, ..., -1.500e+00, -1.869e+00,
        -2.420e+00],
       [-5.273e-01, -2.466e-01, -7.891e-01, ..., -1.221e+00, -8.560e-01,
        -1.460e+00],
       [-1.527e+00, -1.594e+00, -2.480e-01, ...,  1.045e+00, -1.963e+00,
        -1.500e+00]], dtype=float16), label_ids=array([3, 8, 8, ..., 5, 1, 7], dtype=int64), metrics={'test_loss': 1.1890652179718018, 'test_accuracy': 0.764, 'test_runtime': 302.3716, 'test_samples_per_second': 33.072, 'test_steps_per_second': 4.134})